In [1]:
from module_imports import *
from download_data import *
from import_data import *

%matplotlib inline

stock_df, prediction_df = pd.DataFrame(), pd.DataFrame()
pred_tickers = []
source = "Q"
binarize = True
gt = 0
lt = 50.0
vol = 0
if source == "Q":
    stock_df, prediction_df, pred_tickers = get_quandl_data(binarize=binarize, gt=gt, lt=lt, vol=vol)
elif source == "G":
    stock_df, prediction_df = get_goog_data(binarize=binarize, gt=gt, lt=lt, vol=vol)
    
Y = stock_df['label'].values
Y = Y.reshape(Y.shape[0], 1)

X_df = stock_df.drop('label', axis=1)
X = X_df.values

print X.shape, Y.shape
X_df.tail()

(246987, 10) (246987, 1)


,Open,High,Low,Close,Volume,50dravg,200dravg,OC%,HL%,ticker
821,6.90,7.06,6.77,6.97,160500,8.5686,7.04795,0.010145,0.042836,78
822,7.06,7.60,7.03,7.33,175100,8.5522,7.04460,0.038244,0.081081,78
823,7.29,7.60,6.87,6.91,138100,8.5224,7.04130,-0.052126,0.106259,78
824,6.86,6.96,6.56,6.66,121400,8.4868,7.03935,-0.029155,0.060976,78
825,6.68,6.94,6.46,6.56,166700,8.4550,7.03830,-0.017964,0.074303,78


In [2]:
vectorize_label = True
if vectorize_label == True:
    new_y = []
    positives = []
    for i in xrange(Y.shape[0]):
        if Y[i] == 0:
            new_y.append(np.array([[1],[0]]))
        elif Y[i] == 1:
            new_y.append(np.array([[0],[1]]))
    Y = new_y

In [88]:
X_train, X_vt, y_train, y_vt = train_test_split(X, Y, test_size=0.30, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_vt, y_vt, test_size=0.50, random_state=42)

In [91]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

In [92]:
for x,y in zip(X_train, y_train):
    print x.reshape(x.shape[0],1)
    print y.reshape(y.shape[0],1)
    break

[[-0.59493764]
 [-0.59308395]
 [-0.59384637]
 [-0.59969069]
 [-0.08063482]
 [-0.51664036]
 [-0.51954552]
 [-0.17759202]
 [ 0.05011843]
 [ 0.11802   ]]
[[1]
 [0]]


---

In [ ]:
'''
x = np.array([[0],
              [1], 
              [2]])

y = np.array([[0],
              [1]])
'''

features = X_train[0].shape[0]
h1 = 50
h2 = 50
outputs = y_train[0].shape[0]

w1_init = np.sqrt(6)/np.sqrt(h1+features)
W1 = np.random.uniform(low=-w1_init, high=w1_init, size=(h1*features)).reshape(h1,features)
b1 = np.zeros((h1,1))
#print "W1", W1.shape
#print "b1", b1.shape

w2_init = np.sqrt(6)/np.sqrt(h2+h1)
W2 = np.random.uniform(low=-w2_init, high=w2_init, size=(h1*h2)).reshape(h1,h2)
b2 = np.zeros((h2,1))
#print "W2", W2.shape
#print "b2", b2.shape

w3_init = np.sqrt(6)/np.sqrt(outputs+h2)
W3 = np.random.uniform(low=-w3_init, high=w3_init, size=(outputs*h2)).reshape(outputs,h2)
b3 = np.zeros((outputs,1))
#print "W3", W3.shape
#print "b3", b3.shape

#f_x = []

print "-"*10

def loss(f_x,y):
    i = np.where(y == 1)[0][0]
    return -np.log(f_x[i]) # negative log-likelihood

def forward_prop(x, W1, b1, W2, b2, W3, b3):
    def sigm(z):
        return 1/(1+np.exp(-z))
    
    def softmax(z):
        return np.exp(z)/np.sum(np.exp(z))

    z1 = b1 + np.dot(W1,x)
    a1 = sigm(z1)
    
    z2 = b2 + np.dot(W2,a1)
    a2 = sigm(z2)
    
    z3 = b3 + np.dot(W3,a2)
    a3 = softmax(z3)
    
    f_x = a3
    return z1, a1, z2, a2, z3, a3, f_x
'''
z1, a1, z2, a2, z3, a3, f_x = forward_prop(x, W1, b1, W2, b2, W3, b3)
print "y\n", y
print "f_x\n", f_x
print "loss(f_x,y)\n", loss(f_x,y)
print "-"*10
'''

def back_prop(x, W1, b1, W2, b2, W3, b3, z1, a1, z2, a2, z3, a3, f_x, y):
    
    def sigm(z):
        return 1/(1+np.exp(-z))
    
    def sigm_prime(z):
        return (sigm(z) * (1 - sigm(z)))
    
    del_z3 = -(y - f_x)
    del_W3 = np.dot(del_z3,a2.T)
    del_b3 = del_z3
    
    del_a2 = np.dot(W3.T,del_z3)
    del_z2 = np.multiply(del_a2,sigm_prime(z2))
    del_W2 = np.dot(del_z2,a1.T)
    del_b2 = del_z2
    
    del_a1 = np.dot(W2.T,del_z2)
    del_z1 = np.multiply(del_a1,sigm_prime(z1))
    del_W1 = np.dot(del_z1,x.T)
    del_b1 = del_z1
    
    return del_W1, del_b1, del_W2, del_b2, del_W3, del_b3
#del_W1, del_b1, del_W2, del_b2, del_W3, del_b3 = back_prop(x, W1, b1, W2, b2, W3, b3, z1, a1, z2, a2, z3, a3, f_x, y)

def finite_diff_approx(W, b, del_W, del_b, x, f_x, y):
    
    epsilon = 1e-6
    
    # W
    approx_del_W = []
    for i in xrange(W.shape[0]):
        for j in xrange(W.shape[1]):

            temp_w = (W[i][j])
            W[i][j] = W[i][j]+epsilon
            z1, a1, z2, a2, z3, a3, f_x = forward_prop(x, W, b1, W2, b2, W3, b3)
            loss_left = loss(f_x,y)[0]
            W[i][j] = temp_w

            W[i][j] = (W[i][j]-epsilon)
            z1, a1, z2, a2, z3, a3, f_x = forward_prop(x, W, b1, W2, b2, W3, b3)
            loss_right = loss(f_x,y)[0]
            W[i][j] = temp_w
            
            approx_del_W.append((loss_left - loss_right)/(2*epsilon))
            
    print "\nW gradient checking:"
    print "\tapprox_del_W\n\t",approx_del_W[:3]
    print "\tdel_W\n\t",del_W.ravel()[:3]
    print "\tapprox absolute difference:", np.sum(np.abs(approx_del_W - del_W.ravel()))/(len(approx_del_W)**2)
    
    # b
    approx_del_b = []
    for i in xrange(b.shape[0]):
        for j in xrange(b.shape[1]):

            temp_b = b[i][j]
            b[i][j] = (b[i][j]+epsilon)
            z1, a1, z2, a2, z3, a3, f_x = forward_prop(x, W1, b, W2, b2, W3, b3)
            loss_left = loss(f_x,y)[0]
            b[i][j] = temp_b

            b[i][j] = (b[i][j]-epsilon)
            z1, a1, z2, a2, z3, a3, f_x = forward_prop(x, W1, b, W2, b2, W3, b3)
            loss_right = loss(f_x,y)[0]
            b[i][j] = temp_b
            
            approx_del_b.append((loss_left - loss_right)/(2*epsilon))
            
    print "\nb gradient checking:"
    print "\tapprox_del_b\n\t",approx_del_b[:3]
    print "\tdel_b\n\t",del_b.ravel()[:3]
    print "\tapprox absolute difference:", np.sum(np.abs(approx_del_b - del_b.ravel()))/(len(approx_del_b)**2)

#finite_diff_approx(W1, b1, del_W1, del_b1, x, f_x, y)

def regularizer(LN, W):
    if LN == 'L2':
        # np.linalg.norm(W1)**2 + np.linalg.norm(W2)**2 + np.linalg.norm(W3)**2 # L2 regularization
        return (2 * W) # W L2 gradient
    elif LN == 'L1':
        # np.sum(np.abs(W1)) + np.sum(np.abs(W2)) + np.sum(np.abs(W3)) # L1 regularization
        return np.sign(W) # W L1 gradient
    
# SGD
epochs = 10
Lambda = 0.01
LN = 'L2'
alpha = 0.05
#delta = 0.7 # 0.5 < delta <= 1
training_losses = []
validation_losses = []

for i in xrange(epochs):
    
    # training
    training_f_x = []
    training_y = []
    for x,y in zip(X_train, y_train):
        
        x = x.reshape(x.shape[0],1)
        y = y.reshape(y.shape[0],1)

        z1, a1, z2, a2, z3, a3, f_x = forward_prop(x, W1, b1, W2, b2, W3, b3)
        del_W1, del_b1, del_W2, del_b2, del_W3, del_b3 = back_prop(x, W1, b1, W2, b2, W3, b3, z1, a1, z2, a2, z3, a3, f_x, y)
        
        deriv_W3 = -del_W3 - (Lambda * regularizer(LN, W3))
        deriv_b3 = -del_b3
        W3 = W3 + (alpha * deriv_W3)
        b3 = b3 + (alpha * deriv_b3)

        deriv_W2 = -del_W2 - (Lambda * regularizer(LN, W2))
        deriv_b2 = -del_b2
        W2 = W2 + (alpha * deriv_W2)
        b2 = b2 + (alpha * deriv_b2)    

        deriv_W1 = -del_W1 - (Lambda * regularizer(LN, W1))
        deriv_b1 = -del_b1
        W1 = W1 + (alpha * deriv_W1)
        b1 = b1 + (alpha * deriv_b1)
        
        if np.isnan(W1[0])[0] == True:
            raise ValueError('A very specific bad thing happened')
            
        training_f_x = f_x
        training_y = y
    training_losses.append(np.round(loss(training_f_x,training_y),2))
            
    # validation
    validation_f_x = []
    validation_y_v = []
    for x,y in zip(X_validation, y_validation):

        x = scaler.transform(x)
        x = x.reshape(x.shape[0],1)
        y = y.reshape(y.shape[0],1)

        z1, a1, z2, a2, z3, a3, f_x = forward_prop(x, W1, b1, W2, b2, W3, b3)

        if np.isnan(W1[0])[0] == True:
            raise ValueError('A very specific bad thing happened')

        validation_f_x = f_x
        validation_y_v = y_v
    validation_losses.append(np.round(loss(validation_f_x,validation_y_v),2))

    print "current training loss:", training_losses[-1]
    print "current validation loss:", validation_losses[-1]
    plt.title('EPOCH ' + str(i))
    plt.plot(training_losses)
    plt.plot(validation_losses)
    plt.legend(['Training','Validation'])
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    #if i > 5:
    #    alpha = alpha/(1+(delta*i))
    
#z1, a1, z2, a2, z3, a3, f_x = forward_prop(x, W1, b1, W2, b2, W3, b3)
#print "y\n", y
#print "f_x\n", f_x
#print "loss(f_x,y)\n", loss(f_x,y)
#print "-"*10

In [61]:
i = 1400

x_v = X_validation[i].reshape(X_validation[i].shape[0],1)
y_v = y_validation[i].reshape(y_validation[i].shape[0],1)

z1, a1, z2, a2, z3, a3, f_x = forward_prop(x_v, W1, b1, W2, b2, W3, b3)
print "y\n", y_v
print "f_x\n", f_x
print "loss(f_x,y)\n", loss(f_x,y_v)
print "-"*10

y
[[0]
 [1]]
f_x
[[ nan]
 [ nan]]
loss(f_x,y)
[ nan]
----------
